# Advanced Macroeconomics - Heterogenous Agent Models
Assignment I - HANC with two types of labour

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

from HANCModel import HANCModelClass

## a) Setup

In [ ]:
model = HANCModelClass(name='baseline')

In [ ]:
par = model.par
ss = model.ss
path = model.path
sim = model.sim

The DAG for the model

In [ ]:
model.info(only_blocks=True)
model.draw_DAG(figsize=(10,10))

Find the solution of the household problem:

In [ ]:
# guess
ss.r = 0.02
ss.w0 = 1.00
ss.w1 = 1.00
ss.phi0 = 1.0
ss.phi1 = 2.0

In [ ]:
model.solve_hh_ss(do_print=True)
model.simulate_hh_ss(do_print=True)

In [ ]:
#model.test_hh_path()

## b) Solve for the stationary equilibrium

First, I find steady state

In [ ]:
model.find_ss(do_print=True,method='direct')

Check that markets clear:

In [ ]:
model.test_path()

Determinants of wealth inequality

In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

# a. income
ax = fig.add_subplot(1,2,1)
ax.set_title('productivity')
for i_beta,beta in enumerate(par.beta_grid):
    ax.plot(par.z_grid,np.cumsum(np.sum(ss.D[i_beta],axis=1))*par.Nfix,label=f'$\\beta = {beta:.4f}, \\eta = {par.phi_grid[i_beta]:.0f}$')

ax.set_xlabel('productivity, $z_{t}$')
ax.set_ylabel('CDF')
ax.legend()

# b. assets
ax = fig.add_subplot(1,2,2)
ax.set_title('savings')
for i_beta in range(par.Nfix):
    ax.plot(np.insert(par.a_grid,0,par.a_grid[0]),np.insert(np.cumsum(np.sum(ss.D[i_beta],axis=0)),0,0.0)*par.Nfix,label=f'$\\beta = {par.beta_grid[i_beta]:.4f}$')
ax.set_xlabel('assets, $a_{t}$')
ax.set_ylabel('CDF')
ax.set_xscale('symlog')

fig.tight_layout()
fig.savefig('figs/distribution.png')

## c) Compute and inspect the Jacobians of the household block wrt. $\varphi_1$

In [ ]:
model.compute_jacs(do_print=True)

In [ ]:
fig = plt.figure(figsize=(12,len(model.inputs_hh_all)*4),dpi=100)

i = 0
for varname in model.inputs_hh:
    
    # a. savings
    i += 1    
    ax = fig.add_subplot(len(model.inputs_hh_all),2,i)
    ax.set_title(f'savings, $a_t$, to {varname}')
    for s in [0,50,150,250]:
        jac_hh_var = model.jac_hh[('A_hh',varname)]
        ax.plot(np.arange(par.T),jac_hh_var[:,s],label=f'shock at {s}')
    
    if i == 1:
        ax.legend(frameon=True,bbox_to_anchor=(2.75,0.0))

    # b. consumption
    i += 1    
    ax = fig.add_subplot(len(model.inputs_hh_all),2,i)
    ax.set_title(f'consumption, $c_t$, to {varname}')
    for s in [0,50,150,250]:
        jac_hh_var = model.jac_hh[('C_hh',varname)]
        ax.plot(np.arange(par.T),jac_hh_var[:,s],label=f'shock at {s}')
plt.savefig('figs/Jacobians.png')

## d) Solve for the transition path when $\varphi_{1t}$ is 10 percent higher for 10 periods

In [ ]:
model_transitory = model.copy()

In [ ]:
paths = ['phi1','Y','K','r','w0','w1','L0','L1']

# a. define shock
dphi1 = np.zeros(par.T)
dphi1[39:49] = 0.1*par.phi1
shocks = {'dphi1':dphi1}

# b. find transition path
model_transitory.find_transition_path(do_print=True,shocks=shocks)

In [ ]:
model_transitory.show_IRFs(paths,ncols=3,filename='figs/transitory_shock')

In [ ]:
v = np.sum([par.beta_grid[i_fix]**t*np.sum(path.u[t,i_fix]*path.D[t,i_fix]/np.sum(path.D[t,i_fix])) for t in range(par.T)])

## e) Solve for the transition path when $\varphi_{1t}$ is permanently 10 percent higher

In [ ]:
model_permanent = model.copy()

In [ ]:
paths = ['phi1','Y','K','r','w0','w1','L0','L1']

# a. define shock
dphi1 = np.zeros(par.T)
dphi1[20:] = 0.1*par.phi1
shocks = {'dphi1':dphi1}

# b. find transition path
model_permanent.find_transition_path(do_print=True,shocks=shocks)

In [ ]:
model_permanent.show_IRFs(paths,ncols=3,T_max=500,filename='figs/permanent_shock')